# Classificação c/ Naive Bayes

## Importando as bibliotecas

In [1]:
from sklearn.datasets import make_blobs
import numpy as np
from scipy.stats import norm

## Carregando o dataset

In [2]:
X, y = make_blobs(n_samples=100, centers=2, n_features=2, random_state=1)

In [3]:
print(X.shape, y.shape)

(100, 2) (100,)


In [4]:
X

array([[-7.94152277e-01,  2.10495117e+00],
       [-9.15155186e+00, -4.81286449e+00],
       [-3.10367371e+00,  3.90202401e+00],
       [-1.42946517e+00,  5.16850105e+00],
       [-7.46938680e+00, -4.20198333e+00],
       [-9.19585147e+00, -3.90678125e+00],
       [-9.31764266e+00, -4.27315015e+00],
       [-9.20490564e+00, -4.57687928e+00],
       [-2.76017908e+00,  5.55121358e+00],
       [-1.17104176e+00,  4.33091816e+00],
       [-1.00364080e+01, -5.56912090e+00],
       [-1.53773863e+00,  5.53597378e+00],
       [-1.01081595e+01, -4.57071061e+00],
       [-2.40671820e+00,  6.09894447e+00],
       [-7.80701277e+00, -5.84970947e+00],
       [-9.01137732e+00, -3.73981465e+00],
       [-9.76521794e+00, -3.27079714e+00],
       [-2.34673261e+00,  3.56128423e+00],
       [-2.00341358e+00,  4.45008673e+00],
       [-1.10926243e+01, -3.78396611e+00],
       [-9.43495141e+00, -3.71261146e+00],
       [-1.02639310e+01, -3.92073400e+00],
       [-9.74891334e+00, -4.24998970e+00],
       [-1.

In [5]:
y

array([0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1,
       0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1,
       0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0])

## Passo 1: Calculando a probabilidade anterior P(y_i)

In [6]:
def calculate_priors(X, y):
    priors = []
    classes = np.unique(y)
    for target in classes:
        X_target = X[y == target]    
        prior = len(X_target) / len(X)
        priors.append(prior)
    return priors

In [7]:
priors = calculate_priors(X,y)

In [8]:
priors

[0.5, 0.5]

## Passo 2: Gerar as probabilidades condicionais P(x_1|y_i) x P(x_2|y_i) x P(x_3|y_i) x ... x P(x_n|y_i)

In [9]:
def fit_gaussian_distribution(data):
    mu = np.mean(data)
    sigma = np.std(data)
    dist = norm(mu, sigma)
    return dist

In [10]:
# Calculando as PDFs para as características pertencentes a cada 1 das classes
Xy0 = X[y == 0]
Xy1 = X[y == 1]

# Para y=0
X1y0 = fit_gaussian_distribution(Xy0[:, 0]) #P(x_1|0)
X2y0 = fit_gaussian_distribution(Xy0[:, 1]) #P(x_2|0)

# Para y=1
X1y1 = fit_gaussian_distribution(Xy1[:, 0])#P(x_1|1)
X2y1 = fit_gaussian_distribution(Xy1[:, 1]) #P(x_2|1)

## Passo 3: Calcular a Probabilidade Posterior P(y_i| x_1,x_2,...,x_n)

In [37]:
# função que calcula a probabilidade posterior
def posterior_probability(X, prior, dist1, dist2):
    return prior * dist1.pdf(X[0]) * dist2.pdf(X[1])

In [44]:
# calculando a probabilidade condicional para 1 exemplo de entrada
X_test, y_test = X[2], y[2]
posterior = []
Py0 = posterior_probability(X_test, priors[0], X1y0, X2y0)
Py1 = posterior_probability(X_test,  priors[1], X1y1, X2y1)
posterior.append(Py0)
posterior.append(Py1)

In [45]:
print( ' P(y=0 | %s) = %.3f ' % (X_test, Py0*100))
print( ' P(y=1 | %s) = %.3f ' % (X_test, Py1*100))
print( ' Rótulo no dataset: y=%d ' % y_test)

 P(y=0 | [-3.10367371  3.90202401]) = 1.340 
 P(y=1 | [-3.10367371  3.90202401]) = 0.000 
 Rótulo no dataset: y=0 


## Passo 4: Atribuir a amostra à classe com maior probabilidade

In [46]:
y_pred = posterior.index(max(posterior))

In [47]:
y_pred

0

## Parte II: Comparado com a função disponibilizada pelo sklearn

In [48]:
from sklearn.naive_bayes import GaussianNB

In [49]:
# Instanciando o modelo e fitando os dados
model = GaussianNB()
model.fit(X, y)

GaussianNB()

In [51]:
model.get_params()

{'priors': None, 'var_smoothing': 1e-09}

In [2]:
# Realizando a predição 
X_test, y_test = [X[0]], y[0]

NameError: name 'X' is not defined

In [53]:
type(X_test)

list

In [55]:
y_pred = model.predict_proba(X_test)
print( ' Predicted Probabilities: ' , y_pred)
y_pred_class = model.predict(X_test)
print( ' Classe estimada: ' , y_pred_class)
print( ' Rótulo no dataset: y=%d ' % y_test)

 Predicted Probabilities:  [[1.00000000e+00 5.52387327e-30]]
 Classe estimada:  [0]
 Rótulo no dataset: y=0 


Atividade(2.0): Realizar classificação com o Naive Bayes:
<ol>
        <li> Utilizando um dataset que seja multiclasse (a sua escolha). Dica: selecionar no máximo 5 características; </li> 
        <li> Extrapolar o exemplo acima para avaliar a acurácia de classificação para um conjunto de teste (divisão treino/teste a sua escolha);</li> 
        <li> Tentem utilizar uma distribuição diferente da Gaussiana;</li> 
        <li> Comparem a acurácia do modelo desenvolvido por vocês com a acurácia do modelo provido pelo sklearn;</li> 
    
</ol>